In [1]:
# Implement Base class in library.py.

class Base:
    def foo(self):
        return 'foo'

In [3]:
# Implement Derived class in user.py.
# from library import Base

class Derived(Base):
    def Bar(self):
        return self.foo()
    
Derived1 = Derived()
Derived1.Bar()

'foo'

In production, people writing library code know nothing about user code and vice versa. Therefore, problem will arise in calling `self.foo()` if `foo()` is not implmented. To solve the problem, `assert` is needed to ensure the problem can be detected before runtime.

In [4]:
# library.py
class Base:
    pass

# user.py
assert hasattr(Base, 'foo'), 'foo not implemented in the base class.'

class Derived(Base):
    def Bar(self):
        return self.foo()
    
Derived1 = Derived()
Derived1.Bar()

AssertionError: foo not implemented in the base class.

In [6]:
# Python classes are somehow executable.
class Base:
    print('a')

a


Similar problems can occur in the library code if the coders want to make sure that users implement `bar()` method. There are several ways to achieve this goal before runtime.

#### 1. Not a common way by using `__build_class__`

In [7]:
def _():
    class Base:
        pass

from dis import dis
dis(_)

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object Base at 0x7f8a90241a50, file "/var/folders/cv/qql0cpzx0lz5z71s9z0thff80000gn/T/ipykernel_2419/887155197.py", line 2>)
              4 LOAD_CONST               2 ('Base')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('Base')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (Base)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE

Disassembly of <code object Base at 0x7f8a90241a50, file "/var/folders/cv/qql0cpzx0lz5z71s9z0thff80000gn/T/ipykernel_2419/887155197.py", line 2>:
  2           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_CONST               0 ('_.<locals>.Base')
              6 STORE_NAME               2 (__qualname__)

  3           8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


In [1]:
class Base:
    def foo(self):
        return self.bar()

# Rewrite the built-in function.
old_bc = __build_class__
# def my_bc(*a, **kw):
#     print('my buildclass->', a, kw)
#     return old_bc(*a, **kw)
def my_bc(func, name, base=None, **kw):
    print('check if bar method implemented')
    return old_bc(func, name, **kw)
import builtins
builtins.__build_class__ = my_bc

# user.py
class Derived(Base):
    def bar(self):
        return 'bar'

check if bar method implemented


#### 2. `Metaclass` can be used change the `type` of the class. (like a decorator?)

In [6]:
class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if name != 'Base' and not 'bar' in body:
            raise(TypeError('bad user class'))
        return super().__new__(cls, name, bases, body)
    
class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()
    
    def __init_subclass__(self, *a, **kw):
        print('init subclass', a, kw)
        return super().__init_subclass__(*a, **kw)
    
class Derived(Base):
    pass
    # def bar(self):
    #     return 'bar'

TypeError: bad user class

Decorator is a kind of way to avoid writing duplicated codes.

In [7]:
# Bad example: duplicated code
from time import time

def add(x, y=10):
    return x + y

before = time()
add(1, 10)
after = time()
print('time taken:', after - before)

before = time()
add(1)
after = time()
print('time taken:', after - before)

before = time()
add('a', 'b')
after = time()
print('time taken:', after - before)

time taken: 5.602836608886719e-05
time taken: 8.893013000488281e-05
time taken: 5.698204040527344e-05


The very nature of decorator `@timer` is the abbreviation

```
add = timer(add)
```

In [9]:
# Decorator -- a kind of wrapper.
from time import time
def timer(func):
    def f(*args, **kwargs):
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        print('time taken:', after - before)
        return rv 
    return f

@timer
def add(x, y=10):
    return x + y


print(add(1, 10))
print(add(1))
print(add('a', 'b'))

time taken: 1.1920928955078125e-06
11
time taken: 1.9073486328125e-06
11
time taken: 2.1457672119140625e-06
ab


Generator will be able to yield results in advance for the purpose of saving memory and space. Users will not have to wait until the code ends to deal with the results.

More importantly, the generator can be used to ensure that certain functions will be running before certain functions.

Generator can be considered as simplification of a (iterator) class.

In [12]:
# A not so good method is to turn the object into an iterator.
from time import time, sleep

class Compute:
    def __iter__(self):
        self.last = 0
        return self
    
    def __next__(self):
        rv = self.last
        self.last += 1
        if self.last > 10:
            raise StopIteration()
        sleep(.5)
        return rv
    
for val in Compute():
    print(val)

0
1
2
3
4
5
6
7
8
9


In [15]:
# Use generator to achieve the same effect.
def compute():
    for i in range(10):
        sleep(.5)
        yield i

for val in compute():
    print(val)

0
1
2
3
4
5
6
7
8
9


In [ ]:
# Run functiosn in order.
def api():
    first()
    yield
    second()
    yield
    third()
    yield

Context Manager 

Basic context manager application is in file manager. In the following case, the file will be automatically closed and written to disk. Otherwise, if the file is not closed on Windows, the file cannot be deleted. Or there can be risk that the unsaved changes are lost.

In [ ]:
with open ('ctx.py') as f:
    pass

In [18]:
# Another application is in the connection of the database.
from sqlite3 import connect

with connect('test.db') as conn:
    cur = conn.cursor()
    cur.execute('create table points(x int, y int)')
    cur.execute('insert into points (x, y) values (1, 1)')
    cur.execute('insert into points (x, y) values (2, 2)')
    for row in cur.execute('select x, y from points'):
        print(row)
    cur.execute('drop table points')


(1, 1)
(2, 2)


Since each time, create and drop database needs to be called, to simplify the code, we can add a context manager in the following way.

In [25]:
class temptable:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        print("__enter__")
        cur.execute('create table points(x int, y int)')
    def __exit__(self, *args):
        print("__exit__")
        cur.execute('drop table points')

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (2, 2)')
        for row in cur.execute('select x, y from points'):
            print(row)


__enter__
(1, 1)
(2, 2)
__exit__


Because calling `__enter__` and `__exit__` has strict order, it is also possible to simplify the class by using the generator.

In [32]:
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    yield
    cur.execute('drop table points')
    
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

with connect('test.db') as conn:
    cur = conn.cursor()
    with contextmanager(temptable)(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (2, 2)')
        for row in cur.execute('select x, y from points'):
            print(row)  

(1, 1)
(2, 2)


Critically, we can use the decorator to simplify the situation.
Because we have a similar structure to decorator if we set,

```
template = contextmanager(temptable)
```

In [33]:
@contextmanager
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    yield
    cur.execute('drop table points')
    
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (2, 2)')
        for row in cur.execute('select x, y from points'):
            print(row)  

(1, 1)
(2, 2)


But this `contextmanager` has already been implemented in the package `contextlib`.

In [34]:
from sqlite3 import connect
from contextlib import contextmanager

@contextmanager
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    try:
        yield
    finally:
        cur.execute('drop table points')

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values (1, 1)')
        cur.execute('insert into points (x, y) values (2, 2)')
        for row in cur.execute('select x, y from points'):
            print(row)  

(1, 1)
(2, 2)
